In [11]:
import os
from dotenv import load_dotenv
import snowflake.connector
import pandas as pd

In [12]:
# Cargar variables desde el archivo .env
load_dotenv()

# Leer credenciales
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE")  # opcional
)


In [13]:
# Función para ejecutar consultas y devolver resultados como DataFrame
def run_query(query):
    cur = conn.cursor()
    try:
        cur.execute(query)
        df = cur.fetch_pandas_all()
        return df
    finally:
        cur.close()

## Pregunta 1: Demanda por zona y mes: ¿cuáles son las 10 zonas con más viajes por mes? (PU y DO por separado).

In [30]:
query_1 = """
with trips as (
    -- pickups
    select 
        date_trunc('month', d.full_date) as year_month,
        z.zone as zone,
        count(*) as trips_count,
        'PU' as trip_type
    from FCT_TRIPS f
    join DIM_DATE d on f.PICKUP_DATE_SK = d.DATE_SK
    join DIM_ZONE z on f.PU_ZONE_SK = z.ZONE_SK
    where d.full_date between '2009-01-01' and '2025-12-31'
    group by year_month, z.zone

    union all

    -- dropoffs
    select 
        date_trunc('month', d.full_date) as year_month,
        z.zone as zone,
        count(*) as trips_count,
        'DO' as trip_type
    from FCT_TRIPS f
    join DIM_DATE d on f.DROPOFF_DATE_SK = d.DATE_SK
    join DIM_ZONE z on f.DO_ZONE_SK = z.ZONE_SK
    where d.full_date between '2014-12-31' and '2025-08-31'
    group by year_month, z.zone
)
select *
from (
    select 
        year_month,
        trip_type,
        zone,
        trips_count,
        row_number() over (partition by year_month, trip_type order by trips_count desc) as rn
    from trips
) t
where rn <= 10
order by year_month, trip_type, trips_count desc;

"""
df_1 = run_query(query_1)
print("Top 10 zonas con más viajes por mes (Pickup y Dropoff):")
display(df_1)

Top 10 zonas con más viajes por mes (Pickup y Dropoff):


,YEAR_MONTH,TRIP_TYPE,ZONE,TRIPS_COUNT,RN
0,2009-01-01,PU,LaGuardia Airport,26,1
1,2009-01-01,PU,Queensbridge/Ravenswood,15,2
2,2009-01-01,PU,JFK Airport,15,3
3,2009-01-01,PU,Midtown Center,13,4
4,2009-01-01,PU,Upper East Side North,12,5
...,...,...,...,...,...
2606,2025-08-01,PU,Murray Hill,98865,9
2607,2025-08-01,PU,East Village,91675,10
2608,2025-09-01,PU,East Harlem North,12,1
2609,2025-09-01,PU,Manhattanville,1,2


In [28]:
query_2 = """
select max(date_sk), min(date_sk)
from DIM_DATE;
"""
cur = conn.cursor()
cur.execute(query_2)
max_date_sk, min_date_sk = cur.fetchone()
print(f"Rango de DATE_SK en DIM_DATE: {min_date_sk} a {max_date_sk}")

query_2 = """
select count(*) as matched_trips
from FCT_TRIPS f
join DIM_DATE d on f.PICKUP_DATE_SK = d.DATE_SK;
"""
cur.execute(query_2)
matched_trips = cur.fetchone()[0]
print(f"Viajes con fecha de recogida coincidente: {matched_trips}")
cur.close()


Rango de DATE_SK en DIM_DATE: 18991231 a 22530823
Viajes con fecha de recogida coincidente: 852356537


True

## Pregunta 2:

In [16]:
query_2 = """
SELECT 
    dd.year,
    TO_CHAR(dd.full_date, 'Month') AS month,
    dz.borough AS pickup_borough,
    SUM(ft.total_amount) AS total_revenue,
    AVG(CASE WHEN ft.total_amount > 0 THEN ft.tip_amount / ft.total_amount END) AS avg_tip_percentage
FROM gold.fct_trips ft
JOIN gold.dim_date dd ON ft.pickup_date_sk = dd.date_sk
JOIN gold.dim_zone dz ON ft.pu_zone_sk = dz.zone_sk
GROUP BY dd.year, TO_CHAR(dd.full_date, 'Month'), dz.borough
ORDER BY dd.year, TO_CHAR(dd.full_date, 'Month'), dz.borough
"""

df_2 = run_query(query_2)
print("Ingresos totales y porcentaje de propinas por borough y mes:")
display(df_2)

Ingresos totales y porcentaje de propinas por borough y mes:


,YEAR,MONTH,PICKUP_BOROUGH,TOTAL_REVENUE,AVG_TIP_PERCENTAGE
0,2000,Decth,Manhattan,254.70,0.017780
1,2000,Decth,Queens,242.41,0.055556
2,2000,Decth,Unknown,3.80,0.000000
3,2001,Augth,Queens,24.55,0.000000
4,2001,Febth,Queens,3.80,0.000000
...,...,...,...,...,...
1109,2081,Junth,Queens,55.96,0.000000
1110,2084,Novth,Manhattan,73.90,0.000000
1111,2088,Janth,Manhattan,20.60,0.000000
1112,2090,Decth,Queens,61.42,0.000000
